In [8]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [16]:
subject = 'Bahia - Consumo de Cimento (t)'
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [17]:
data = pd.read_csv('2003_mo_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego,Bahia - Consumo de Cimento (t)
0,2003-1,0.299858,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.669899,39.798880,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,8.348779,151.297
1,2003-2,0.301903,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.670210,39.480034,1.318964e+08,8.391946e+06,8.569210,1.216914e+08,8.342979,138.707
2,2003-3,0.303709,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.670521,39.400256,1.320584e+08,8.399299e+06,8.572270,1.217469e+08,8.337179,135.009
3,2003-4,0.305311,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.670831,39.417185,1.322204e+08,8.406652e+06,8.575331,1.218023e+08,8.331379,126.554
4,2003-5,0.306860,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.671142,39.479943,1.323824e+08,8.414005e+06,8.578392,1.218578e+08,8.325579,137.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.597113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.069163,NaN,NaN,NaN,NaN,NaN,366.305
236,2022-9,0.596178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.752943,NaN,NaN,NaN,NaN,NaN,346.042
237,2022-10,0.594662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.537361,NaN,NaN,NaN,NaN,NaN,347.901
238,2022-11,0.592436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.971241,NaN,NaN,NaN,NaN,NaN,310.845


In [18]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.436732,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-2.195846,-1.793459,-1.766933,-0.695996,-2.315693,-2.238131,-0.944845
1,-1.404185,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-2.161313,-1.806616,-1.747868,-0.654222,-2.273375,-2.196868,-0.946500
2,-1.375442,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-2.126781,-1.809908,-1.728803,-0.612447,-2.231056,-2.155605,-0.948156
3,-1.349941,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-2.092248,-1.809210,-1.709739,-0.570673,-2.188738,-2.114342,-0.949811
4,-1.325290,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-2.057715,-1.806620,-1.690674,-0.528898,-2.146420,-2.073079,-0.951467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.851312,-0.689886,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.041952,0.619029,0.865987,-1.416165,0.555618,0.463875,1.107985
200,1.873607,-0.441954,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.026783,0.635352,0.848481,-1.404857,0.531641,0.443334,1.106555
201,1.901718,-0.132782,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.011615,0.652040,0.830975,-1.393550,0.507664,0.422793,1.105126
202,1.928136,0.084061,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,0.996447,0.638794,0.813469,-1.382243,0.483687,0.402252,1.103697


In [19]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      138.707
1      135.009
2      126.554
3      137.331
4      118.680
        ...   
235    346.042
236    347.901
237    310.845
238    310.845
239        NaN
Name: Bahia - Consumo de Cimento (t), Length: 240, dtype: float64

In [20]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.436732,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-2.195846,-1.793459,-1.766933,-0.695996,-2.315693,-2.238131,-0.944845
1,-1.404185,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-2.161313,-1.806616,-1.747868,-0.654222,-2.273375,-2.196868,-0.946500
2,-1.375442,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-2.126781,-1.809908,-1.728803,-0.612447,-2.231056,-2.155605,-0.948156
3,-1.349941,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-2.092248,-1.809210,-1.709739,-0.570673,-2.188738,-2.114342,-0.949811
4,-1.325290,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-2.057715,-1.806620,-1.690674,-0.528898,-2.146420,-2.073079,-0.951467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.005306,-0.742853,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806,1.339087,0.469766,1.163205,-1.340370,1.005749,1.008566,1.160245
164,1.029532,-0.787367,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523,1.336451,0.457435,1.164562,-1.364998,1.008863,1.002613,1.155001
165,1.057198,-0.757940,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175,1.333816,0.454716,1.165920,-1.389626,1.011977,0.996661,1.149757
166,1.082730,-0.917469,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146,1.331180,0.446427,1.167277,-1.414254,1.015092,0.990708,1.144512


In [21]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      138.707
1      135.009
2      126.554
3      137.331
4      118.680
        ...   
163    280.074
164    277.938
165    275.802
166    273.666
167    271.530
Name: Bahia - Consumo de Cimento (t), Length: 168, dtype: float64

In [28]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
168,1.142199,-1.032458,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117,1.325909,0.407445,1.169992,-1.463511,1.021321,0.978804,1.134024
169,1.169669,-1.077854,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274,1.318823,0.400025,1.166430,-1.470219,1.023238,0.968444,1.133050
170,1.197459,-1.036724,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341,1.311738,0.398398,1.162868,-1.476927,1.025155,0.958085,1.132076
171,1.231686,-1.013400,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301,1.304652,0.379182,1.159306,-1.483635,1.027072,0.947726,1.131102
172,1.268038,-1.059228,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284,1.297567,0.384352,1.155744,-1.490343,1.028989,0.937367,1.130127
173,1.299588,-1.170032,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348,1.290482,0.367059,1.152182,-1.497051,1.030906,0.927007,1.129153
174,1.338328,-1.115621,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659,1.283396,0.363065,1.148620,-1.503759,1.032823,0.916648,1.128179
175,1.376421,-1.182159,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184,1.276311,0.360415,1.145058,-1.510467,1.034740,0.906289,1.127205
176,1.409717,-1.270623,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173,1.269226,0.359113,1.141496,-1.517175,1.036658,0.895930,1.126231
177,1.441251,-1.321931,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760,1.262140,0.367682,1.137934,-1.523883,1.038575,0.885571,1.125257


In [29]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168    213.266
169    285.938
170    219.576
171    267.203
172    240.714
173    250.101
174    277.528
175    270.092
176    278.146
177    257.458
178    231.748
179    268.336
180    223.453
181    241.464
182    238.901
183    191.989
184    272.452
185    261.009
186    292.688
187    258.881
188    276.879
189    255.774
190    208.326
191    291.428
192    249.430
193    241.612
194    252.303
195    281.912
196    200.213
197    270.511
198    281.466
199    285.535
200    328.259
201    298.078
202    264.838
203    300.311
Name: Bahia - Consumo de Cimento (t), dtype: float64

In [30]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [37]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=100)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [38]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][100:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [39]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[93470216, 1864188991, 2125944638, 3839403813, 1099321741, 2105300297, 4114045829, 2208471635, 2119684419, 3278814380, 3039031841, 1811316228, 1080030228, 1631001836, 2725908411, 2701734392, 2865431260, 900984803, 4221230527, 4078569769, 4057710384, 2770560984, 857218811, 1866134998, 3942350069]


Step: 0 ___________________________________________
val_loss: 168.2988739013672
winner_seed: 93470216


Step: 1 ___________________________________________
val_loss: 156.1582794189453
winner_seed: 1864188991


Step: 2 ___________________________________________
val_loss: 109.13557434082031
winner_seed: 2125944638


Step: 3 ___________________________________________
val_loss: 154.4282684326172


Step: 4 ___________________________________________
val_loss: 189.16162109375


Step: 5 ___________________________________________
val_loss: 147.6299591064453


Step: 6 ___________________________________________
val_loss: 114.7603759765625


Step: 7 ___________________________________________
val_lo

In [40]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 21ms/step - loss: 53349.0625 - val_loss: 46122.2812
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 34046.9062 - val_loss: 34689.5625
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 25713.4883 - val_loss: 13351.8643
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 21137.0859 - val_loss: 13606.6387
Epoch 5/10000
5/5 [==============================] - 0s 5ms/step - loss: 9991.1172 - val_loss: 9505.7197
Epoch 6/10000
5/5 [==============================] - 0s 5ms/step - loss: 4972.8989 - val_loss: 1701.9733
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 1107.8701 - val_loss: 562.3380
Epoch 8/10000
5/5 [==============================] - 0s 4ms/step - loss: 615.9182 - val_loss: 384.4833
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 433.6516 - val_loss: 689.4791
Epoch 10/10000
5/5 [==============================]

5/5 [==============================] - 0s 4ms/step - loss: 401.7391 - val_loss: 362.8146
Epoch 158/10000
5/5 [==============================] - 0s 4ms/step - loss: 393.0506 - val_loss: 345.3229
Epoch 159/10000
5/5 [==============================] - 0s 4ms/step - loss: 470.8242 - val_loss: 353.3344
Epoch 160/10000
5/5 [==============================] - 0s 4ms/step - loss: 389.1624 - val_loss: 272.0139
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 341.9490 - val_loss: 241.5444
Epoch 162/10000
5/5 [==============================] - 0s 4ms/step - loss: 341.0694 - val_loss: 274.1913
Epoch 163/10000
5/5 [==============================] - 0s 4ms/step - loss: 444.3780 - val_loss: 260.5183
Epoch 164/10000
5/5 [==============================] - 0s 4ms/step - loss: 552.8311 - val_loss: 561.3952
Epoch 165/10000
5/5 [==============================] - 0s 4ms/step - loss: 471.2847 - val_loss: 387.5493
Epoch 166/10000
5/5 [==============================] - 0s 4ms/step - lo

5/5 [==============================] - 0s 4ms/step - loss: 327.8161 - val_loss: 253.8775
Epoch 314/10000
5/5 [==============================] - 0s 4ms/step - loss: 415.0000 - val_loss: 371.4319
Epoch 315/10000
5/5 [==============================] - 0s 4ms/step - loss: 367.3710 - val_loss: 534.8921
Epoch 316/10000
5/5 [==============================] - 0s 4ms/step - loss: 451.6684 - val_loss: 303.4141
Epoch 317/10000
5/5 [==============================] - 0s 4ms/step - loss: 408.6630 - val_loss: 340.8770
Epoch 318/10000
5/5 [==============================] - 0s 4ms/step - loss: 455.9684 - val_loss: 291.4306
Epoch 319/10000
5/5 [==============================] - 0s 4ms/step - loss: 359.3570 - val_loss: 378.0467
Epoch 320/10000
5/5 [==============================] - 0s 4ms/step - loss: 450.3416 - val_loss: 437.6800
Epoch 321/10000
5/5 [==============================] - 0s 4ms/step - loss: 418.2013 - val_loss: 234.7765
Epoch 322/10000
5/5 [==============================] - 0s 4ms/step - lo

5/5 [==============================] - 0s 4ms/step - loss: 310.4743 - val_loss: 248.8380
Epoch 470/10000
5/5 [==============================] - 0s 4ms/step - loss: 321.8301 - val_loss: 279.8866
Epoch 471/10000
5/5 [==============================] - 0s 4ms/step - loss: 360.4780 - val_loss: 265.8479
Epoch 472/10000
5/5 [==============================] - 0s 4ms/step - loss: 395.4560 - val_loss: 321.3398
Epoch 473/10000
5/5 [==============================] - 0s 4ms/step - loss: 353.3319 - val_loss: 265.8326
Epoch 474/10000
5/5 [==============================] - 0s 4ms/step - loss: 414.5211 - val_loss: 320.8477
Epoch 475/10000
5/5 [==============================] - 0s 5ms/step - loss: 342.3692 - val_loss: 266.9911
Epoch 476/10000
5/5 [==============================] - 0s 4ms/step - loss: 368.0693 - val_loss: 271.3084
Epoch 477/10000
5/5 [==============================] - 0s 4ms/step - loss: 363.4303 - val_loss: 253.8668
Epoch 478/10000
5/5 [==============================] - 0s 4ms/step - lo

5/5 [==============================] - 0s 4ms/step - loss: 349.1040 - val_loss: 295.2705
Epoch 626/10000
5/5 [==============================] - 0s 4ms/step - loss: 357.8526 - val_loss: 237.3129
Epoch 627/10000
5/5 [==============================] - 0s 4ms/step - loss: 427.1756 - val_loss: 374.3631
Epoch 628/10000
5/5 [==============================] - 0s 4ms/step - loss: 441.7744 - val_loss: 170.0813
Epoch 629/10000
5/5 [==============================] - 0s 4ms/step - loss: 330.7119 - val_loss: 251.4628
Epoch 630/10000
5/5 [==============================] - 0s 4ms/step - loss: 309.1832 - val_loss: 281.4476
Epoch 631/10000
5/5 [==============================] - 0s 5ms/step - loss: 342.1815 - val_loss: 278.4582
Epoch 632/10000
5/5 [==============================] - 0s 4ms/step - loss: 306.8509 - val_loss: 263.8978
Epoch 633/10000
5/5 [==============================] - 0s 4ms/step - loss: 327.0013 - val_loss: 310.0683
Epoch 634/10000
5/5 [==============================] - 0s 4ms/step - lo

5/5 [==============================] - 0s 4ms/step - loss: 358.2167 - val_loss: 191.4508
Epoch 782/10000
5/5 [==============================] - 0s 4ms/step - loss: 341.7480 - val_loss: 335.8157
Epoch 783/10000
5/5 [==============================] - 0s 4ms/step - loss: 361.9315 - val_loss: 229.5339
Epoch 784/10000
5/5 [==============================] - 0s 4ms/step - loss: 362.7165 - val_loss: 313.7154
Epoch 785/10000
5/5 [==============================] - 0s 4ms/step - loss: 348.3308 - val_loss: 248.5162
Epoch 786/10000
5/5 [==============================] - 0s 4ms/step - loss: 359.7555 - val_loss: 258.0753
Epoch 787/10000
5/5 [==============================] - 0s 4ms/step - loss: 365.6752 - val_loss: 243.1444
Epoch 788/10000
5/5 [==============================] - 0s 4ms/step - loss: 362.2953 - val_loss: 344.2623
Epoch 789/10000
5/5 [==============================] - 0s 4ms/step - loss: 372.9055 - val_loss: 287.8242
Epoch 790/10000
5/5 [==============================] - 0s 4ms/step - lo

In [41]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [42]:
errors, mae, mape = mae_mape_calculator(trained_model, test_input, test_target, 168)

1/1 [==============================] - 0s 50ms/step
Month-1 - Error: [[64.80965]]
1/1 [==============================] - 0s 13ms/step
Month-2 - Error: [[7.8657837]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[58.49611]]
1/1 [==============================] - 0s 13ms/step
Month-4 - Error: [[10.869293]]
1/1 [==============================] - 0s 13ms/step
Month-5 - Error: [[37.35756]]
1/1 [==============================] - 0s 13ms/step
Month-6 - Error: [[27.974869]]
1/1 [==============================] - 0s 13ms/step
Month-7 - Error: [[5.062378]]
1/1 [==============================] - 0s 13ms/step
Month-8 - Error: [[22.417236]]
1/1 [==============================] - 0s 13ms/step
Month-9 - Error: [[14.459473]]
1/1 [==============================] - 0s 13ms/step
Month-10 - Error: [[35.19406]]
1/1 [==============================] - 0s 13ms/step
Month-11 - Error: [[60.893997]]
1/1 [==============================] - 0s 13ms/step
Month-12 - Error: [[18.251923]]
1/1 [=

In [43]:
display(mae)
display(mape)

184.54974

0.70035625

In [44]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [45]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[3408.0269]] - Target[3060.1059999999998]| =  Error: [[347.9209]]; MAPE:[[0.1136957]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[1457.3331]] - Target[3013.2439999999997]| =  Error: [[1555.9108]]; MAPE:[[0.5163574]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[-997.43365]] - Target[3254.4680000000003]| =  Error: [[4251.902]]; MAPE:[[1.3064814]]


[array([[347.9209]], dtype=float32),
 array([[1555.9108]], dtype=float32),
 array([[4251.902]], dtype=float32)]

2051.9111

0.64551145

## 